# Tune GPT2 to generate positive reviews
> Optimise GPT2 to produce positive IMDB movie reviews using a BERT sentiment classifier for rewards.

<div style="text-align: center">
<img src='images/gpt2_bert_training.png' width='600'>
<p style="text-align: center;"> <b>Figure:</b> Experiment setup to tune GPT2. The yellow arrows are outside the scope of this notebook, but the trained models are available through Hugging Face. </p>
</div>


In this notebook we fine-tune GPT2 (small) to generate positive movie reviews based on the IMDB dataset. The model gets 5 tokens from a real review and is tasked to produce positive continuations. To reward positive continuations we use a BERT classifier to analyse the sentiment of the produced sentences and use the classifier's outputs as rewards signals for PPO training.

## Setup experiment

### Import dependencies

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import wandb
import time
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
tqdm.pandas()

from datasets import load_dataset

from transformers import GPT2Tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from trl.gpt2 import GPT2HeadWithValueModel, respond_to_batch
from trl.ppo import PPOTrainer
from trl.core import build_bert_batch_from_txt

### Configuration

In [ ]:
config = {
    "lm_name": "lvwerra/gpt2-imdb",
    "ref_lm_name": "lvwerra/gpt2-imdb",
    "cls_model_name": "lvwerra/distilbert-imdb",
    "tk_name": "gpt2",
    "steps": 25600,
    "batch_size": 256,
    "forward_batch_size": 16,
    "ppo_epochs": 4,   
    "txt_in_len": 5,
    "txt_out_len": 15,
    "lr": 1.41e-5,
    "init_kl_coef":0.2,
    "target": 6,
    "horizon":10000,
    "gamma":1,
    "lam":0.95,
    "cliprange": .2,
    "cliprange_value":.2,
    "vf_coef":.1, 
}

You can see that we load a GPT2 model called `gpt2_imdb`. This model was additionally fine-tuned on the IMDB dataset for 1 epoch with the huggingface [script](https://github.com/huggingface/transformers/blob/master/examples/run_language_modeling.py) (no special settings). The other parameters are mostly taken from the original paper ["Fine-Tuning Language Models from Human Preferences"](
https://arxiv.org/pdf/1909.08593.pdf). This model as well as the BERT model is available in the Huggingface model zoo [here](https://huggingface.co/models). The following code should automatically download the models.

### Initialize W&B logger
We use `wandb`to log all the metrics during training.

In [ ]:
wandb.init(name='run-42', project='gpt2-sentiment', config=config, )

## Load data and models

### Load IMDB dataset
The IMDB dataset contains 50k movie review annotated with "positive"/"negative" feedback indicating the sentiment.  We load the IMDB dataset into a DataFrame and filter for comments that are at least 500 characters long and take the first 1000 characters of each comment. The first filter we apply to avoid comments that are less than `txt_in_len` token long and the second to avoid tokenizing way more text than we actually need.

In [ ]:
# load imdb with datasets
ds = load_dataset('imdb', split='train')
ds = ds.rename_columns({'text': 'review', 'label': 'sentiment'})
ds.set_format('pandas')
df = ds[:]

# make sure the comments are long enough
df = df.loc[df['review'].str.len() > 500]

# make sure comments are not too long
df['review'] = df['review'].apply(lambda x: x[:1000])

df.head()

Reusing dataset imdb (/Users/leandro/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


,review,sentiment
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


### Load BERT classifier
We load a BERT classifier fine-tuned on the IMDB dataset.

In [ ]:
sentiment_model = AutoModelForSequenceClassification.from_pretrained(config["cls_model_name"])
sentiment_tokenizer = AutoTokenizer.from_pretrained(config["cls_model_name"])

The model outputs are the logits for the negative and positive class. We will use the logits for positive class as a reward signal for the language model.

In [ ]:
text = 'this movie was really bad!!'
output = sentiment_model.forward(**sentiment_tokenizer(text, return_tensors="pt"))
output

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.3350, -2.7266]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
text = 'this movie was really good!!'
output = sentiment_model.forward(**sentiment_tokenizer(text, return_tensors="pt"))
output

SequenceClassifierOutput(loss=None, logits=tensor([[-2.2948,  2.5570]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

The resulting reward signal:

In [ ]:
output.logits[0, 1]

tensor(2.5570, grad_fn=<SelectBackward0>)

### Load pre-trained GPT2 language models

We load the GPT2 model with a value head and the tokenizer. We load the model twice; the first model is optimized while the second model serves as a reference to calculate the KL-divergence from the starting point. This serves as an additional reward signal in the PPO training to make sure the optimized model does not deviate too much from the original language model.

In [ ]:
gpt2_model = GPT2HeadWithValueModel.from_pretrained(config['lm_name'])
gpt2_model_ref = GPT2HeadWithValueModel.from_pretrained(config['ref_lm_name'])
gpt2_tokenizer = GPT2Tokenizer.from_pretrained(config['tk_name'])

### Watch model with wandb
This wandb magic logs the gradients and weights of the model during training.

In [ ]:
wandb.watch(gpt2_model, log='all')

### Move models to GPU

If `cuda` is available move the computations to the GPU.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
_ = gpt2_model.to(device)
_ = sentiment_model.to(device)
_ = gpt2_model_ref.to(device)

### Tokenize IMDB reviews

We tokenize all IMDB in advance to avoid tokenizing twice. In the first step we encode the queries and slice the first `txt_in_len` tokens. In a second step we decode these tokens back to text for later display.

In [ ]:
df['tokens'] = df['review'].progress_apply(lambda x: gpt2_tokenizer.encode(x, return_tensors="pt").to(device)[0, :config['txt_in_len']])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22578/22578 [00:21<00:00, 1054.55it/s]


In [ ]:
df['query'] = df['tokens'].progress_apply(lambda x: gpt2_tokenizer.decode(x))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22578/22578 [00:00<00:00, 93906.80it/s]


## Optimize model

**Steps**

The training loop consists of the following steps:
1. Get a batch of queries
2. Get the query responses from the policy
3. Join query and responses and tokenize for BERT analysis
4. Get sentiments for query/responses from BERT
5. Optimize policy with PPO using the (query, response, reward) triplet
6. Log all the training statistics

**Forward batching**

Since the models can be fairly big and we want to rollout large PPO batches this can lead to out-of-memory errors when doing the forward passes for text generation and sentiment analysis. We introduce the parameter `forward_batch_size` to split the forward passes into smaller batches. Although this hurts performance a little this is neglectible compared to the computations of the backward passes when optimizing the model. The same parameter is used in the `PPOTrainer` when doing forward passes. The `batch_size` should multiple of `forward_batch_size`.

**Training time**

This step takes **~2h** on a P6000 GPU with the above specified settings.

In [ ]:
def listify_batch(tensor):
    return [tensor[i] for i in range(tensor.shape[0])]

In [ ]:
ppo_trainer = PPOTrainer(gpt2_model, gpt2_model_ref, gpt2_tokenizer, **config)
fbs = config['forward_batch_size']

for epoch in tqdm(range(int(np.ceil(config["steps"]/config['batch_size'])))):
    torch.cuda.empty_cache()
    logs = dict()
    game_data = dict()
    timing = dict()
    t0 = time.time()
    
    #### get a batch from the dataset
    df_batch = df.sample(config['batch_size'])
    game_data['query'] = df_batch['query'].tolist()
    query_tensors = torch.stack(df_batch['tokens'].tolist())
    
    #### get response from gpt2
    t = time.time()
    total_length = config['txt_in_len']+config['txt_out_len']
    response_tensors = []
    for i in range(int(config['batch_size']/fbs)):
        response  = respond_to_batch(gpt2_model, query_tensors[i*fbs:(i+1)*fbs],
                                     txt_len=config['txt_out_len'])
        response_tensors.append(response)
    response_tensors = torch.cat(response_tensors)
    game_data['response'] = [gpt2_tokenizer.decode(response_tensors[i, :]) for i in range(config['batch_size'])]
    timing['time/get_response'] = time.time()-t

    #### tokenize text for sentiment analysis
    t = time.time()
    texts = [q + r for q,r in zip(game_data['query'], game_data['response'])]
    sentiment_inputs, attention_masks = build_bert_batch_from_txt(texts, sentiment_tokenizer, device)    
    timing['time/build_input_sentiment'] = time.time()-t

    #### get sentiment score
    t = time.time()
    rewards = []
    for i in range(int(config['batch_size']/fbs)):
        res = sentiment_model.forward(sentiment_inputs[i*fbs:(i+1)*fbs],
                                      attention_masks[i*fbs:(i+1)*fbs])[0][:, 1].detach()
        rewards.append(res)
    rewards = torch.cat(rewards)
    timing['time/get_sentiment_preds'] = time.time()-t
    
    #### Run PPO training 
    t = time.time()
    stats = ppo_trainer.step(listify_batch(query_tensors), listify_batch(response_tensors), listify_batch(rewards))
    timing['time/optimization'] = time.time()-t
     
    #### Log everything
    timing['time/epoch'] = time.time()-t0
    table_rows = [list(r) for r in zip(game_data['query'], game_data['response'], rewards.cpu().tolist())]
    logs.update({'game_log':wandb.Table(
        columns=['query', 'response', 'reward'],
        rows=table_rows)})
    logs.update(timing)
    logs.update(stats)
    logs['env/reward_mean'] = torch.mean(rewards).cpu().numpy()
    logs['env/reward_std'] = torch.std(rewards).cpu().numpy()
    logs['env/reward_dist'] = rewards.cpu().numpy()
    wandb.log(logs)

  0%|                                                                                                                                                                                        | 0/6400 [00:00<?, ?it/s]/Users/leandro/git/trl/env/lib/python3.9/site-packages/torch/nn/modules/module.py:990: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/Users/leandro/git/trl/env/lib/python3.9/site-packages/torch/nn/modules/module.py:1025: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get 

torch.Size([15]) torch.Size([15]) torch.Size([15])
kl tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) logprob tensor([-1.9644e+00, -5.9458e+00, -1.7350e-01, -9.0757e-01, -7.1213e+00,
        -3.2678e+00, -1.2245e+00, -4.1328e+00, -1.8766e+00, -1.0494e-03,
        -1.1888e+00, -1.1470e+00, -7.0350e+00, -3.3745e+00, -8.2286e-01]) ref_logprob tensor([-1.9644e+00, -5.9458e+00, -1.7350e-01, -9.0757e-01, -7.1213e+00,
        -3.2678e+00, -1.2245e+00, -4.1328e+00, -1.8766e+00, -1.0494e-03,
        -1.1888e+00, -1.1470e+00, -7.0350e+00, -3.3745e+00, -8.2286e-01])
kl tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]) logprob tensor([ -5.2349,  -4.1780,  -6.1149,  -2.5166,  -1.1029,  -4.3439,  -2.8021,
         -2.1040,  -3.4833,  -1.7423,  -7.1941,  -0.0906, -11.0389,  -1.6325,
         -1.3833]) ref_logprob tensor([ -5.2349,  -4.1780,  -6.1149,  -2.5166,  -1.1029,  -4.3439,  -2.8021,
         -2.1040,  -3.4833,  -1.7423,  -7.1941,  -0.0906, -11.0389,  -1.

  0%|                                                                                                                                                                             | 1/6400 [00:19<34:15:26, 19.27s/it]

torch.Size([15]) torch.Size([15]) torch.Size([15])
kl tensor([-6.4710e-02, -7.4575e-02, -8.7449e-02,  2.2474e-01, -7.4406e-02,
         1.4406e-01,  3.8504e-01, -1.4087e-01,  2.3842e-07, -5.9604e-07,
         0.0000e+00, -1.1921e-07,  0.0000e+00, -5.1203e-02, -9.3641e-02]) logprob tensor([-2.1130e+00, -3.5285e+00, -5.6247e+00, -1.9345e+00, -3.0730e+00,
        -1.9022e+00, -2.6603e+00, -3.4323e+00, -8.4638e-06, -6.4373e-06,
        -1.1921e-07, -7.1525e-06, -3.5763e-07, -2.0676e+00, -3.8449e+00]) ref_logprob tensor([-2.0483e+00, -3.4540e+00, -5.5373e+00, -2.1593e+00, -2.9986e+00,
        -2.0463e+00, -3.0453e+00, -3.2915e+00, -8.7022e-06, -5.8412e-06,
        -1.1921e-07, -7.0333e-06, -3.5763e-07, -2.0164e+00, -3.7513e+00])
kl tensor([-1.1871e-02, -2.6386e-02,  9.4759e-02,  2.3766e-01,  1.2494e-02,
         2.3921e-02, -2.9335e-01,  5.3885e-02,  8.8508e-02, -4.5478e-02,
        -1.9683e-01,  5.5154e-05,  6.6463e-02, -3.2894e-02,  6.8202e-02]) logprob tensor([-9.1770e-01, -9.6816e-01, -

  0%|                                                                                                                                                                             | 2/6400 [00:38<34:10:11, 19.23s/it]

...

torch.Size([15]) torch.Size([15]) torch.Size([15])
kl tensor([ 8.6001e-01,  2.0639e+00, -3.6726e-01,  8.5437e-01,  4.7723e-01,
         7.0566e-01, -7.3709e-01, -1.2769e+00,  1.3635e+00, -9.4533e-04,
        -2.9711e-01,  3.5186e-01,  8.3771e-01, -2.0158e-02,  1.2197e+00]) logprob tensor([ -5.8461,  -7.1563,  -5.1163,  -3.9668,  -9.4803, -10.4818,  -4.9550,
         -6.9482,  -3.4138,  -0.0357,  -7.5852,  -7.2133,  -5.6429,  -0.1453,
         -4.8735]) ref_logprob tensor([ -6.7061,  -9.2202,  -4.7490,  -4.8212,  -9.9575, -11.1875,  -4.2179,
         -5.6713,  -4.7773,  -0.0347,  -7.2881,  -7.5652,  -6.4806,  -0.1251,
         -6.0933])
kl tensor([ 0.0851,  0.2502,  0.1594,  2.4001,  0.1737, -0.3213,  0.5438,  0.6576,
         0.0214, -2.0009, -0.3536, -0.5231,  2.2169, -0.3809,  0.3581]) logprob tensor([ -3.3662,  -6.0903,  -3.0321,  -6.0927,  -0.3710, -10.5381,  -2.0044,
         -6.4298,  -0.0140,  -6.5035,  -2.1593,  -5.6143,  -6.0114,  -6.5348,
         -2.0977]) ref_logprob tensor

  1%|█▋                                                                                                                                                                          | 61/6400 [19:33<33:55:29, 19.27s/it]

torch.Size([15]) torch.Size([15]) torch.Size([15])
kl tensor([ 0.0081, -0.1295,  0.9265, -0.2469,  0.0348,  0.0953,  0.6344,  1.3417,
         0.6624,  0.7137,  0.7834,  0.3338,  0.3198,  1.3267,  0.7414]) logprob tensor([-3.8832, -0.1324, -4.6387, -2.5551, -2.1391, -0.4132, -2.0345, -2.1618,
        -1.4940, -3.0288, -3.3465, -9.0780, -1.2464, -5.2797, -3.4243]) ref_logprob tensor([-3.8913e+00, -2.9438e-03, -5.5653e+00, -2.3081e+00, -2.1739e+00,
        -5.0851e-01, -2.6689e+00, -3.5035e+00, -2.1565e+00, -3.7425e+00,
        -4.1299e+00, -9.4117e+00, -1.5662e+00, -6.6063e+00, -4.1658e+00])
kl tensor([ 0.5062,  0.0348, -0.1326,  0.5689, -0.5341,  1.1539, -0.4501, -0.1212,
         0.1031, -0.7759,  0.7290, -0.0962, -0.5873,  0.4556,  0.5830]) logprob tensor([-3.6407, -0.1028, -5.1355, -5.3064, -4.5646, -1.9301, -1.0590, -0.5856,
        -0.5349, -5.6912, -4.9315, -0.1940, -1.4914, -1.6462, -0.7168]) ref_logprob tensor([-4.1469, -0.1375, -5.0029, -5.8752, -4.0305, -3.0840, -0.6088, -0.4

  1%|█▋                                                                                                                                                                          | 62/6400 [19:57<34:00:35, 19.32s/it]


KeyboardInterrupt: 

### Training progress
If you are tracking the training progress with Weights&Biases you should see a plot similar to the one below. Check out the interactive sample report on wandb.ai: [link](https://app.wandb.ai/lvwerra/trl-showcase/runs/1jtvxb1m/).

<div style="text-align: center">
<img src='images/gpt2_tuning_progress.png' width='800'>
<p style="text-align: center;"> <b>Figure:</b> Reward mean and distribution evolution during training. </p>
</div>

One can observe how the model starts to generate more positive outputs after a few optimisation steps.

> Note: Investigating the KL-divergence will probably show that at this point the model has not converged to the target KL-divergence, yet. To get there would require longer training or starting with a higher inital coefficient.

## Model inspection
Let's inspect some examples from the IMDB dataset. We can use `gpt2_model_ref` to compare the tuned model `gpt2_model` against the model before optimisation.

In [ ]:
#### get a batch from the dataset
bs = 16
game_data = dict()
df_batch = df.sample(bs)
game_data['query'] = df_batch['query'].tolist()
query_tensors = torch.stack(df_batch['tokens'].tolist())

#### get response from gpt2 and gpt2_ref
total_length = config['txt_in_len']+config['txt_out_len']
response_tensors_ref  = respond_to_batch(gpt2_model_ref, query_tensors, txt_len=config['txt_out_len'])
game_data['response (before)'] = [gpt2_tokenizer.decode(response_tensors_ref[i, :]) for i in range(bs)]

response_tensors  = respond_to_batch(gpt2_model, query_tensors, txt_len=config['txt_out_len'])
game_data['response (after)'] = [gpt2_tokenizer.decode(response_tensors[i, :]) for i in range(bs)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q,r in zip(game_data['query'], game_data['response (before)'])]
sentiment_inputs, attention_masks = build_bert_batch_from_txt(texts, sentiment_tokenizer, device)    
rewards = sentiment_model.forward(sentiment_inputs, attention_masks)[0][:, 1].detach()
game_data['rewards (before)'] = rewards.cpu().numpy()

texts = [q + r for q,r in zip(game_data['query'], game_data['response (after)'])]
sentiment_inputs, attention_masks = build_bert_batch_from_txt(texts, sentiment_tokenizer, device)    
rewards = sentiment_model.forward(sentiment_inputs, attention_masks)[0][:, 1].detach()
game_data['rewards (after)'] = rewards.cpu().numpy()

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

,query,response (before),response (after),rewards (before),rewards (after)
0,That's what the title,said to me. All I had to do was hear one word...,"says, and this movie was very good,-I really ...",1.710360,4.207656
1,The female hot numbers on,"the Fonda get a saving grace here, you can st...",the DVD are beautiful. The music is extraordi...,1.585194,4.228330
2,In the film Kongwon,"(American Truffaut), a snake made of ash fall...",has a funky personality and makes the movies ...,-0.015413,2.537073
3,Let's see: what,"a blast from start to finish, female and male...",happens on that later?<|endoftext|> This film...,3.278678,4.061680
4,Really? Is this necessary,for Sith philosophy combined with missionary ...,"? Watched this too, but it's what made it so b...",-2.509291,3.025524
5,During a sleepless night,"Keira Knightley has besieged a monastery, tro...",", I found this Parker Bros. film. The acting i...",1.341672,3.933312
6,My wife spotted this film,"almost halfway through second grade, and afte...",when I was younger and kept her P-1-her-p-,1.781651,1.456647
7,I love John's work,but I actually have to write language as in w...,and I hereby recommend this film. I am really...,-1.904006,4.159198
8,I's a big struggle,to see anyone who acts in that way. by Jim Th...,", but overall I'm happy with the changes even ...",-1.595925,2.651260
9,i liked this film a,lot Paul Verhoeven The other actor does what ...,lot recipe. It was probably one of the best m...,4.314877,4.346435


Looking at the reward mean/median of the generated sequences we observe a significant difference.

In [ ]:
print('mean:')
display(df_results.mean())
print()
print('median:')
display(df_results.median())

mean:


rewards (before)    0.211196
rewards (after)     3.068743
dtype: float64


median:


rewards (before)    0.296938
rewards (after)     3.997496
dtype: float64

## Save model
Finally, we save the model to disk for later usage.

In [ ]:
os.makedirs('gpt2-imdb-pos')
gpt2_model.save_pretrained('gpt2-imdb-pos')
gpt2_tokenizer.save_pretrained('gpt2-imdb-pos')

('gpt2-imdb-pos/vocab.json',
 'gpt2-imdb-pos/merges.txt',
 'gpt2-imdb-pos/special_tokens_map.json',
 'gpt2-imdb-pos/added_tokens.json')